In [109]:
from ortools.linear_solver import pywraplp
from datetime import datetime, date
import pandas as pd
import calendar
solver = pywraplp.Solver.CreateSolver('GLOP')

## Variáveis do problema

In [110]:
month_idx = 11
now = datetime.now()
n_days = calendar.monthrange(now.year, month_idx)[1]
weekdays  = [date(now.year, month_idx, d).weekday() for d in range(1, n_days + 1)]
first_weekday = weekdays[0]
print(len(calendar.monthcalendar(now.year, month_idx)))

5


In [111]:
residents = [
    { "idx": 0, "name": "André", "type": "R1" },
    { "idx": 1, "name": "Maria", "type": "R1" },
    { "idx": 2, "name": "Cinthya", "type": "R1" },
    { "idx": 3, "name": "Rolando", "type": "R1" },
    { "idx": 4, "name": "Victor", "type": "R1" },
    { "idx": 5, "name": "Lucas", "type": "R2" },
    { "idx": 6, "name": "Giselia", "type": "R2" },
    { "idx": 7, "name": "Pedro", "type": "R2" },
    { "idx": 8, "name": "Bárbara", "type": "R2" },
    { "idx": 9, "name": "Filipe", "type": "R3" },
]

n_res = len(residents)

In [112]:
enf_list = ['André', 'Maria', 'Cinthya', 'Rolando', 'Victor']
enf_list = [r for r in residents if r['name'] in enf_list]
enf_list

[{'idx': 0, 'name': 'André', 'type': 'R1'},
 {'idx': 1, 'name': 'Maria', 'type': 'R1'},
 {'idx': 2, 'name': 'Cinthya', 'type': 'R1'},
 {'idx': 3, 'name': 'Rolando', 'type': 'R1'},
 {'idx': 4, 'name': 'Victor', 'type': 'R1'}]

In [113]:
shifts = [
    { "idx": 0, "name": "HRC_dia", "start_time": 7, "end_time": 19, "frequency": [1,1,1,1,1,1,1], "num_res": 2 },
    { "idx": 1, "name": "HRC_noite", "start_time": 19, "end_time": 7, "frequency": [1,1,1,1,1,1,1], "num_res": 2 },
    { "idx": 2, "name": "ENF", "start_time": 7, "end_time": 19, "frequency": [1,1,1,1,1,1,1], "num_res": 2 },
    { "idx": 3, "name": "CC", "start_time": 7, "end_time": 19, "frequency": [1,1,1,1,1,0,0], "num_res": 2 },
    { "idx": 4, "name": "HRSam", "start_time": 19, "end_time": 24, "frequency": [1,1,1,0,0,0,0], "num_res": 2 },
    { "idx": 5, "name": "Amb", "start_time": 7, "end_time": 19, "frequency": [1,1,1,1,1,0,0], "num_res": 1 },
]

n_shifts = len(shifts)

In [114]:
def convert_date(hour, day=1):
    return datetime.now().replace(day=day, hour=hour-1, minute=0, second=0, microsecond=0)

def get_time(shift):
    start = shift['start_time']
    end = shift['end_time']
    if end < start:
        return convert_date(start), convert_date(end, 2)
    return convert_date(start), convert_date(end)

def check_overlap(s1, s2):
    start1, end1 = get_time(s1)
    start2, end2 = get_time(s2)
    if end1 > start2 and end2 > start1:
        return 1

    return 0

overlaps = [[check_overlap(s1, s2) for s1 in shifts] for s2 in shifts]


In [115]:
for i, shift in enumerate(shifts):
    start, end = get_time(shift)
    shifts[i]['workload'] = (end - start).seconds//3600

## Variável alvo

In [116]:
Xrsd = [[[solver.BoolVar(f'X{r}{s}{d}') for d in range(n_days)] for s in range(n_shifts)] for r in range(n_res)]

## Restrições

1. Garantir que tenha o número de residentes necessário em cada plantão

In [117]:
for s, shift in enumerate(shifts):
    for d, weekday in enumerate(weekdays):

        # Find the number of residents depending on weekday
        num_res = 0
        if shift['frequency'][weekday]:
            num_res = shift['num_res']

        ct = solver.Constraint(num_res, num_res, f"Day {d + 1} - {num_res} residents in {shift['name']}")
        soma = ""
        for r in range(n_res):
            ct.SetCoefficient(Xrsd[r][s][d], 1)

2. Garantir que não haja choque de horário na escala

In [118]:
for d in range(n_days):
    for r, resident in enumerate(residents):
        for shift_idx, shift in enumerate(shifts):
            ct = solver.Constraint(0, 1, f"No time overlap in day {d + 1} for resident {resident['name']} - Shift {shift['name']}")
            for s, overlap in enumerate(overlaps[shift_idx]):
                ct.SetCoefficient(Xrsd[r][s][d], overlap)

3. Ambulatório só tem R1

In [119]:
s = 5
shift = shifts[s]
r_mais = [res for res in residents if res['type'] in ["R2", "R3"]]

for d in range(n_days):
    for resident in r_mais:
        r = resident['idx']
        ct = solver.Constraint(0, 0, f"No r+ in ambulatory - day {d + 1} - resident {resident['name']}")
        ct.SetCoefficient(Xrsd[r][s][d], 1)

4. HRC_dia/noite, HRSam, ENF e CC precisa de um R1 e um R2/R3 por turno

In [120]:
for d in range(n_days):
    for s, shift in enumerate(shifts[:-1]):
        ct = solver.Constraint(0, 0, f"R1+R2/R3 in day {d + 1} - shift {shift['name']}")
        for r, resident in enumerate(residents):
            is_r1 = 1 if resident['type'] == 'R1' else -1
            ct.SetCoefficient(Xrsd[r][s][d], is_r1)

5. R3 faz HRC_dia ou CC 1x por semana cada

In [121]:
r3_shifts = [shifts[i] for i in [0, 3]]
r3_list = [res for res in residents if res['type']  == "R3"]

for resident in r3_list:
    r = resident['idx']
    for shift in r3_shifts:
        s = shift['idx']
        week_counter = 0
        for d, weekday in enumerate(weekdays):
            if weekday == first_weekday:
                week_counter += 1
                ct = solver.Constraint(1, 1, f"R3 {resident['name']} working once in shift {shift['name']} - week {week_counter}")
            ct.SetCoefficient(Xrsd[r][s][d], 1)

6. R3 não faz parte dos demais plantões (HRC_noite, amb, enf, hrsam)

In [122]:
not_r3_shifts = [shifts[i] for i in [1, 2, 4, 5]]

ct = solver.Constraint(0, 0, f"R3 residents doesnt work in other shifts")
for resident in r3_list:
    r = resident['idx']
    for shift in not_r3_shifts:
        s = shift['idx']
        for d in range(n_days):
            ct.SetCoefficient(Xrsd[r][s][d], 1)

7. O R1 fica a semana toda na enfermaria

In [123]:
week_counter = 0
s = 2  # ENF

for d, weekday in enumerate(weekdays):
    if weekday == first_weekday:
        resident = enf_list[week_counter]
        r = resident['idx']
        week_counter += 1
    ct = solver.Constraint(1, 1, f"R1 {resident['name']} working on ENF - week {week_counter + 1}")
    ct.SetCoefficient(Xrsd[r][s][d], 1)

8. Máximo de horas de trabalho semanais por residente

In [124]:
max_workload = 110

for r, resident in enumerate(residents):
    week_counter = 0
    for d, weekday in enumerate(weekdays):

        if weekday == first_weekday:
            ct = solver.Constraint(0, max_workload, f"R1 {resident['name']} working between {0} and {max_workload} hors - week {week_counter + 1}")
            week_counter += 1

        for s, shift in enumerate(shifts):
            ct.SetCoefficient(Xrsd[r][s][d], shift['workload'])

9. Máximo 24 horas seguidas de trabalho

In [125]:
# como implementar?

## Função objetivo

In [126]:
objective = solver.Objective()

for d in range(n_days):
    for s in range(n_shifts):
        for r in range(n_res):
            objective.SetCoefficient(Xrsd[r][s][d], 1)

objective.SetMinimization()

In [127]:
solver.NumConstraints()

2371

In [128]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('OTIMA')

OTIMA


In [129]:
r = 0
resident = residents[r]

# for r, resident in enumerate(residents):
print(resident['name'])

for d in range(n_days):
    for s, shift in enumerate(shifts[:-1]):
        if round(Xrsd[r][s][d].solution_value(), 0):
            print(f"DIA {d + 1} - {shift['name']}")

André
DIA 1 - HRC_noite
DIA 1 - ENF
DIA 2 - ENF
DIA 2 - HRSam
DIA 3 - HRC_noite
DIA 3 - ENF
DIA 4 - ENF
DIA 5 - ENF
DIA 6 - ENF
DIA 7 - ENF
DIA 8 - HRSam
DIA 9 - HRSam
DIA 10 - HRC_dia
DIA 10 - HRC_noite
DIA 12 - CC
DIA 13 - HRC_dia
DIA 14 - HRC_dia
DIA 15 - HRSam
DIA 16 - HRC_dia
DIA 16 - HRC_noite
DIA 17 - HRC_dia
DIA 17 - HRSam
DIA 18 - CC
DIA 19 - HRC_noite
DIA 19 - CC
DIA 21 - HRC_dia
DIA 21 - HRC_noite
DIA 22 - HRSam
DIA 23 - HRC_noite
DIA 24 - HRC_noite
DIA 25 - HRC_noite
DIA 25 - CC
DIA 26 - HRC_dia
DIA 26 - HRC_noite
DIA 27 - HRC_noite
DIA 28 - HRC_dia
DIA 28 - HRC_noite


In [130]:
week_counter = 0

for r, resident in enumerate(residents):
    if week_counter:
        print(f"Semana {week_counter}: {soma}")
    print(f"\n\n RESIDENTE {resident['name']}")

    week_counter = 0
    soma = 0

    for d, weekday in enumerate(weekdays):

        if weekday == first_weekday:
            if week_counter:
                print(f"Semana {week_counter}: {soma}")
            soma = 0
            week_counter += 1

        for s, shift in enumerate(shifts):
            soma += round(Xrsd[r][s][d].solution_value(), 0) * shift['workload']



 RESIDENTE André
Semana 1: 113.0
Semana 2: 106.0
Semana 3: 106.0
Semana 4: 113.0
Semana 5: 12.0


 RESIDENTE Maria
Semana 1: 108.0
Semana 2: 96.0
Semana 3: 17.0
Semana 4: 77.0
Semana 5: 17.0


 RESIDENTE Cinthya
Semana 1: 10.0
Semana 2: 65.0
Semana 3: 84.0
Semana 4: 36.0
Semana 5: 29.0


 RESIDENTE Rolando
Semana 1: 60.0
Semana 2: 60.0
Semana 3: 96.0
Semana 4: 89.0
Semana 5: 24.0


 RESIDENTE Victor
Semana 1: 96.0
Semana 2: 60.0
Semana 3: 84.0
Semana 4: 72.0
Semana 5: 48.0


 RESIDENTE Lucas
Semana 1: 72.0
Semana 2: 77.0
Semana 3: 72.0
Semana 4: 89.0
Semana 5: 41.0


 RESIDENTE Giselia
Semana 1: 82.0
Semana 2: 108.0
Semana 3: 101.0
Semana 4: 72.0
Semana 5: 0.0


 RESIDENTE Pedro
Semana 1: 60.0
Semana 2: 65.0
Semana 3: 24.0
Semana 4: 113.0
Semana 5: 29.0


 RESIDENTE Bárbara
Semana 1: 89.0
Semana 2: 53.0
Semana 3: 106.0
Semana 4: 29.0
Semana 5: 12.0


 RESIDENTE Filipe
Semana 1: 24.0
Semana 2: 24.0
Semana 3: 24.0
Semana 4: 24.0


In [131]:
schedule = []

for d, weekday in enumerate(weekdays):
    day_schedule = []
    for s, shift in enumerate(shifts):
        res_list = []
        for r, resident in enumerate(residents):
            if round(Xrsd[r][s][d].solution_value(), 0):
                res_list.append(resident['name'])
        day_schedule.append(', '.join(res_list))
    schedule.append(day_schedule)

In [132]:
cols = [s['name'] for s in shifts]
idx = [f'{d + 1}/{month_idx}' for d in range(n_days)]


pd.DataFrame(schedule, columns=cols, index=idx)

,HRC_dia,HRC_noite,ENF,CC,HRSam,Amb
1/11,"Maria, Lucas","André, Giselia","André, Giselia","Rolando, Bárbara","Cinthya, Bárbara",Victor
2/11,"Maria, Pedro","Maria, Pedro","André, Bárbara","Rolando, Lucas","André, Giselia",Victor
3/11,"Rolando, Bárbara","André, Bárbara","André, Pedro","Maria, Giselia","Cinthya, Giselia",Victor
4/11,"Rolando, Lucas","Victor, Bárbara","André, Pedro","Maria, Filipe",,Victor
5/11,"Rolando, Filipe","Maria, Bárbara","André, Lucas","Maria, Giselia",,Victor
6/11,"Victor, Giselia","Victor, Lucas","André, Bárbara",,,
7/11,"Maria, Giselia","Maria, Lucas","André, Pedro",,,
8/11,"Victor, Giselia","Maria, Giselia","Maria, Bárbara","Rolando, Filipe","André, Pedro",André
9/11,"Cinthya, Filipe","Rolando, Giselia","Maria, Lucas","Rolando, Giselia","André, Lucas",André
10/11,"André, Pedro","André, Lucas","Maria, Lucas","Cinthya, Giselia","Cinthya, Bárbara",Victor
